# Deep Learning Lab: ECE-00450107
## Meeting 2 - Part 1: Convolutional Networks

Before running the code in this file, make sure that you are **activating the enviourment** in which the following packages are installed.

#### Definitions and Imports:

In [1]:
from torch.utils.data import DataLoader 
from collections import Counter
from typing import List, Optional, Any, Tuple, Dict
from DL_Lab2_functions import *
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
matplotlib.use('Agg')
warnings.filterwarnings('ignore')
%matplotlib tk

## Deep Convolutional Network (using VGG13), Classifing CIFAR10

In this meeting we will implement and train a deep convolutional network, to classify the CIFAR10 dataset.
CIFAR 10, as the name suggests, contains 10 classes of objects. Here are some sample images:
<center><img src="assets/cifar10.jpeg" width="400px"></center>

So... Let us start :-)

#### Set fixed seeds to enable reproducing the results:

In [2]:
torch.cuda.empty_cache()

# Setting fixed seeds
seed = 2025
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = False

#### Loading the Training Data
load the dataset and calculate mean and standard deviation.

In [3]:
train = torchvision.datasets.CIFAR10(root="/usr/share/DL_exp/datasets/cifar10", train=True, download=True)
test = torchvision.datasets.CIFAR10(root="/usr/share/DL_exp/datasets/cifar10", train=False, download=True)
class_names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
train_size = len(train)
test_size = len(test)

# Count the number of samples in each category
print(f"The train set contains {train_size} images, divided into {len(class_names)} sections:")
labels = [label for _, label in train]
label_counts = Counter(labels)
for label, count in label_counts.items():
    print(f"Label {label}: {count} examples")

print(f"The test set contains {test_size} images, divided into {len(class_names)} sections:")
labels = [label for _, label in test]
label_counts = Counter(labels)
for label, count in label_counts.items():
    print(f"Label {label}: {count} examples")

Files already downloaded and verified
Files already downloaded and verified
The train set contains 50000 images, divided into 10 sections:
Label 6: 5000 examples
Label 9: 5000 examples
Label 4: 5000 examples
Label 1: 5000 examples
Label 2: 5000 examples
Label 7: 5000 examples
Label 8: 5000 examples
Label 3: 5000 examples
Label 5: 5000 examples
Label 0: 5000 examples
The test set contains 10000 images, divided into 10 sections:
Label 3: 1000 examples
Label 8: 1000 examples
Label 0: 1000 examples
Label 6: 1000 examples
Label 1: 1000 examples
Label 9: 1000 examples
Label 5: 1000 examples
Label 7: 1000 examples
Label 4: 1000 examples
Label 2: 1000 examples


In [4]:
train_data = torch.tensor(train.data).to(float())/255
test_data = torch.tensor(test.data).to(float())/255

mean_imn = torch.mean(train_data, dim=(0, 1, 2))
std_imn = torch.std(train_data, dim=(0, 1, 2))
print(f"mean: {[round(value, 3) for value in mean_imn.tolist()]}, std: {[round(value, 3) for value in std_imn.tolist()]}") 

mean: [0.491, 0.482, 0.447], std: [0.247, 0.243, 0.262]


#### Setting the Test/Train Transformations
Here we define the transformations that will be applied to the images during training and testing.

In [5]:
use_augmentataions = True
if use_augmentataions:
    augmentations = [transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
                    transforms.RandomHorizontalFlip()]
    train.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_imn, std=std_imn)] + augmentations)
    test.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_imn, std=std_imn)])
else:
    train.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_imn, std=std_imn)])
    test.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean_imn, std=std_imn)])

#### Defining a VGG Block
Define a single VGG block according to the specifications in the booklet.

In [6]:
class VGGBlock(nn.Module):
    def __init__(self, c_in: int, c_out: int):
        super(VGGBlock, self).__init__()
        self.block = nn.Sequential(nn.Conv2d(in_channels=c_in,out_channels=c_in,kernel_size=3,padding=1),  
                                   nn.BatchNorm2d(num_features=c_in), # TODO: complete a batch normalization layer
                                   nn.ReLU(), # TODO: complete an activation layer
                                   nn.Conv2d(in_channels=c_in,out_channels=c_out,kernel_size=3,padding=1), # TODO: complete a convolution layer with kernel size 3x3 and padding size of 1
                                   nn.BatchNorm2d(num_features=c_out), # TODO: complete a batch normalization layer
                                   nn.ReLU(), # TODO: complete an activation layer
                                   nn.MaxPool2d(kernel_size=2,stride=2)  # TODO: complete a max pooling layer with kernal size 2 and stride 2
                                   )

    def forward(self, x):
        return self.block(x)

#### Defining the VGG Network
Now, we can define the entire network, based on blocks.  
We will use 5 intermediate blocks, add a dropout layer, and lastly add a fully-connected layer at the end to classify the images.  

In [7]:
class VGG13(nn.Module):
    def __init__(self, input_channels: Tuple[int, ...], mid_channels: Tuple[int, ...], output_channels: int, p: float = 0):
        super(VGG13, self).__init__()
        self.vggblocks = nn.Sequential(VGGBlock(input_channels[0], mid_channels[0]),
                                       VGGBlock(mid_channels[0], mid_channels[1]),
                                       VGGBlock(mid_channels[1], mid_channels[2]),
                                       VGGBlock(mid_channels[2], mid_channels[3]),
                                       VGGBlock(mid_channels[3], mid_channels[4]))

        # TODO: Define a dropout layer with probability p
        self.dropout = nn.Dropout2d(p=p)
        # TODO: Define the final linear layer
        self.linear = nn.Linear(in_features=mid_channels[4],out_features=output_channels)
        # TODO: Define a flatten layer
        self.flatten = nn.Flatten()
        # TODO: Define a softmax layer
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        # TODO: Complete the vggblocks sequence
        z1 = self.vggblocks(x)
        # TODO: Complete the dropout layer
        z2 = self.dropout(z1)
        # TODO: Complete the flatten of z2
        z3 =  self.flatten(z2)
        # TODO: Complete the final linear layer
        z4 = self.linear(z3)
        # TODO: Add softmax in the end
        out = self.softmax(z4)
        return out

#### Defining the Hyper-parameters
Define the hyper-parameters for the training of the VGG13 network according to the specifications in the booklet.

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu" #Set device

hparams = Hyper_Params()

hparams.epochs = 40
hparams.batch_size = 250

hparams.lr = 0.001
hparams.dropout_probability = 0
hparams.scheduler_step_size = 8
hparams.scheduler_factor = 0.5


#### Setting Up the Training Parameters
Define the VGG13 model using the parameters specified in the booklet.
In addition, define the scheduler, the optimizer and the loss function.

In [9]:
model = VGG13(input_channels=(3, 32, 32),
              mid_channels=[64, 128, 256, 512, 1024], # complete the values of the channels according to the network's definition in the booklet
              output_channels=10,
              p=hparams.dropout_probability).to(device) 

# TODO: Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=hparams.lr)
# TODO: Define the learning rate scheduler of type StepLR
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=hparams.scheduler_step_size, gamma=hparams.scheduler_factor)
# TODO: Define the loss function using cross entropy
loss_function = nn.CrossEntropyLoss()

# Define dataloaders for the train and test sets: we shall set the num_workers to 16 and use shuffle for training
train_loader = DataLoader(train, batch_size=hparams.batch_size, shuffle=True, num_workers=16)
test_loader = DataLoader(test, batch_size=hparams.batch_size, shuffle=False, num_workers=16)

#### Defining a Training Loop
Complete the training loop.

In [10]:
def train_with_scheduler(hparams: Hyper_Params,
                         train_loader: DataLoader,
                         test_loader: DataLoader,
                         model: nn.Module,
                         optimizer: optim.Optimizer,
                         scheduler: optim.lr_scheduler.LRScheduler,
                         loss_function: nn.Module) -> None:
    hparams.fig, (hparams.ax1, hparams.ax2) = plt.subplots(2, 1, figsize=(15, 9))
    print_performance_grid(Flag=True)
    iter_num = len(train_loader)

    # Set the model to training mode
    model.train()
    start_time = time.time() # time the start of training    
     # Training loop
    for epoch in range(hparams.epochs):      
        hparams.epoch_accuracy_train = np.zeros(len(train_loader))
        hparams.epoch_loss_train = np.zeros(len(train_loader))
      
        for i, batch in enumerate(train_loader):
            # get a new batch of images and labels
            data, target = batch
            target = target.reshape(target.shape[0]).to(device)

            # forward pass
            output = model(data.to(device))
            # TODO: Use the loss function on the output and target to get the train loss
            loss = loss_function(output, target)

            # backward pass
            # TODO: reset the optimizer's gradients
            optimizer.zero_grad()
            # TODO: complete the backward
            loss.backward()
            # TODO: complete the algorithm step
            optimizer.step()
               
            # save the loss and accuracy for the graph visualization
            hparams.epoch_accuracy_train[i] = multi_class_accuracy(output, target)
            hparams.epoch_loss_train[i] = loss.item()
                
            if (i == 0 and epoch == 0) or ((i+1) == iter_num):
                torch.cuda.empty_cache()
                model.eval()
                test_loss, test_accuracy = evaluate(test_loader, model, loss_function)
                print_performance(epoch, i, hparams, test_loss, test_accuracy)
                model.train()

            # TODO: Position A - should the scheduler.step() be here? Or...
            # ----- end of batch loop -----
        scheduler.step()
        # TODO: Position B - should the scheduler.step() be here?
        # ----- end of epoch loop -----

    plt.show()
    print(f"Total training took {time.time() - start_time:.2f} seconds")    

Defining the evaluation function is the same as before.

In [11]:
# Evaluation function
def evaluate(test_loader, model, loss_function):
    with torch.no_grad():
        loss = 0
        acc = 0
        for i, batch in enumerate(test_loader):
            data, target = batch
            output = model(data.to(device))
            batch_loss = loss_function(output, target.to(device))
            batch_acc = multi_class_accuracy(output, target.to(device))
            loss+= batch_loss
            acc+=batch_acc
        return loss.item()/len(test_loader),acc/len(test_loader)

#### Train the Model
Run the training loop and plot the loss and accuracy of the model on the training and test sets.

In [12]:
# Training
train_with_scheduler(hparams=hparams, train_loader=train_loader, test_loader=test_loader,
                     model=model, optimizer=optimizer, scheduler=scheduler, loss_function=loss_function)
present_confusion_matrix(model, test_loader, class_names, device=device)

| Epoch No. | Iter No. | Train Loss | Train Accuracy | Test Loss | Test Accuracy |
----------------------------------------------------------------------------------
|     0     |    1     |    2.31    |      0.08      |    2.3    |      0.1      |
----------------------------------------------------------------------------------
|     0     |   200    |    2.14    |      0.32      |   2.08    |     0.38      |
----------------------------------------------------------------------------------
|     1     |   200    |    2.04    |      0.42      |    2.0    |     0.46      |
----------------------------------------------------------------------------------
|     2     |   200    |    1.98    |      0.47      |    2.0    |     0.46      |
----------------------------------------------------------------------------------
|     3     |   200    |    1.96    |      0.49      |   1.95    |     0.51      |
----------------------------------------------------------------------------------
|   